In [ ]:
# The old notebook became too confusing!

# Trying to check Singular Values and Singular Vectors in Numpy
 
import numpy as np

# General Set-up
N = 2
A = 0.1 * np.random.randn(N, N) + np.diag(np.arange(1, N+1))
B = np.random.randn(N, N)
I = np.eye(N)
dA = np.random.randn(N, N)
dB = np.random.randn(N, N)
bC = np.random.randn(N, N)
eps = 1e-20
epsi = 1 / eps
Ae = A + 1j*eps*dA
Be = B + 1j*eps*dB # don't need it, I'm working with single input.

# SVD Set-up
u, s, vT = np.linalg.svd(A)

# Complex part
De, Du = np.linalg.eig(Ae.dot(Ae.T))
Du, _, _ = np.linalg.svd(Ae)
idx = De.argsort()[::-1]   
De = De[idx]
Du = u[:,idx]
D = np.real(De) # only needed for what? 

# forward propagation 
dA = dA
dS = np.diag(I * u.T.dot(dA).dot(vT.T))

E = np.outer(np.ones(N), s) - np.outer(s, np.ones(N))
F = 1 / (E + np.eye(N)) - np.eye(N)
dU = u.dot(F * (u.T.dot(dA).dot(vT.T).dot(np.diag(s)) + np.diag(s).dot(vT).dot(dA.T).dot(u)))

# backward gradients
bS = np.random.randn(N) # gradients wrt singular values
bA = u.dot(np.diag(bS)).dot(vT) # backpropagated gradient wrt to matrix A

print('singular value')
print('svd error: ', (np.linalg.norm(s-np.sqrt(D))))

# Forward Check based on complex matrices
print('CVT error (vals): ', (np.linalg.norm(2*s*dS - epsi*np.imag(De))))
print('CVT error (vecs): ', (np.linalg.norm(dU - epsi*np.imag(Du))))

# Backward Check, these are essentially two traces!!
print('adj error: ',np.sum(dA*bA)-np.sum(dS*bS))

# I should be able to use the same identity for my testing purposes!!!
# trace(bC.T * dC) = trace(bA.T * dA) + trace(bB.T * dB)



In [ ]:
import numpy as np

# Checking eigenvectors and eigenvalues in Numpy
np.random.seed(10)
# General Set-up
N = 5
A = 0.1 * np.random.randn(N, N) + np.diag(np.arange(1, N+1))
#A = A.dot(A.T)
B = np.random.randn(N, N)
I = np.eye(N)
dA = np.random.randn(N, N)
dB = np.random.randn(N, N)
bC = np.random.randn(N, N)
eps = 1e-20
epsi = 1 / eps
Ae = A + 1j*eps*dA
Be = B + 1j*eps*dB

# EIGEN Set-up
De, Ue = np.linalg.eig(Ae)
D = np.real(De)
U = np.real(Ue)

# make dC diagonal equal to zero
Ue = Ue.dot(np.diag(1 / np.diag(np.linalg.inv(U).dot(Ue))))
E = np.outer(np.ones(N), D) - np.outer(D, np.ones(N))
F = 1 / (E + np.eye(N)) - np.eye(N)
P = np.linalg.inv(U).dot(dA.dot(U))
dD = np.eye(N) * P
dU = U.dot(F*P)
#dD, dU = forward(A, dA)

bD = np.diag(np.random.randn(N)) # random perturbation of eigenvalues
bU = np.random.randn(N,N) # random perturbation of eigenvectors
bD = bD + F * (U.T.dot(bU))
bA = np.linalg.inv(U.T).dot(bD.dot(U.T))
print('eigenvalues and eigenvectors')
print('CVT error (vals): ', np.linalg.norm(np.diag(dD) - epsi*np.imag(De)))
print('CVT error (vecs): ', np.linalg.norm(dU - epsi*np.imag(Ue)))
print('adj error (backward): ', np.sum(dA*bA)-np.sum(dD*bD)-np.sum(dU*bU))
#print('CVT error (vecs): ', np.linalg.norm(np.abs(dU) - np.abs(epsi*np.imag(Ue))))

In [ ]:
dU

In [ ]:
# essentials for eig in PyTorch
import torch

N = 5
A = torch.randn(N,N).double()
L = A.mm(A.t())
I = torch.eye(N).double()
dA = torch.randn(N, N).double() 
dL = dA.mm(A.t()) + A.mm(dA.t())

vals, vecs = torch.eig(L, eigenvectors=True)
vals = vals[:,0]
E = vals.expand(N,N) - vals.expand(N,N).t()
F = 1 / (E + I) - I
P = torch.inverse(vecs).mm(dL.mm(vecs))
dvals = I * P
dvecs = vecs.mm(F*P)

# backward pass
bvals = torch.randn(N).diag().double()
bvecs = torch.randn(N,N).double()
med = bvals + F * (vecs.t().mm(bvecs))
bL = torch.inverse(vecs.t()).mm(med.mm(vecs.t()))
bA = bL.mm(A) + bL.t().mm(A)

# check
torch.sum(bA * dA) - torch.sum(dvals * bvals) - torch.sum(dvecs * bvecs)

In [3]:
# essentials for svd in PyTorch

import torch

M = 6
N = 5
A = torch.randn(M,N).double()
dA = torch.randn(M, N).double() 
dL = dA.mm(A.t()) + A.mm(dA.t())

vecs, vals, v = torch.svd(A)
vals = vals**2
s = vals.size(0)
I = torch.eye(s).double()
E = vals.expand(s,s) - vals.expand(s,s).t()
F = 1 / (E + I) - I
P = vecs.t().mm(dL.mm(vecs))
dvals = I * P
dvecs = vecs.mm(F*P)

# backward pass
bvals = torch.randn(s).diag().double()
bvecs = torch.randn(vecs.size()).double()
med = bvals + F * (vecs.t().mm(bvecs))
bL = vecs.mm(med.mm(vecs.t()))
bA = bL.mm(A) + bL.t().mm(A)

# check
torch.sum(bA * dA) - torch.sum(dvals * bvals) - torch.sum(dvecs * bvecs)

8.881784197001252e-16

In [4]:
from torch.autograd import Variable
from dpp_nets.my_torch.linalg import custom_decomp

A = Variable(A, requires_grad=True)
vals, vecs = custom_decomp()(A)
torch.autograd.backward([vals, vecs],[bvals.diag(), bvecs])
torch.sum(A.grad.data == bA)

30